In [9]:
import os
from tqdm import tqdm
import glob
import random

In [7]:
import pandas as pd
import tensorflow as tf

name_dict = dict(
    inputs=[
        # Your Inputs
    ],
    targets=[
        # Your Targets For Inputs Respectively.
    ]
)

In [10]:
summary_files = glob.glob('../val_2005/summary/*')

In [31]:
summary_files[:10]

['../val_2005/summary/US20050225040A1_summary',
 '../val_2005/summary/US20050149233A1_summary',
 '../val_2005/summary/US20050034453A1_summary',
 '../val_2005/summary/US20050073128A1_summary',
 '../val_2005/summary/US20050243340A1_summary',
 '../val_2005/summary/US20050083025A1_summary',
 '../val_2005/summary/US20050158169A1_summary',
 '../val_2005/summary/US20050007372A1_summary',
 '../val_2005/summary/US20050118919A1_summary',
 '../val_2005/summary/US20050103926A1_summary']

In [11]:
count = 0
for summary_file in summary_files:
    
    filepath_elements = summary_file.split('/')
    pa_id = filepath_elements[3].split('_')[0]
    
    with open(summary_file, "r") as file:
        # Read each line in the file, readlines() returns a list of lines
        summary = file.readlines()
        # Combine the lines in the list into a string
        summary = "".join(summary)
        if (len(summary.split())) > 800:
            continue
    
    firstclaim_file = '../val_2005/firstclaim/' + pa_id + '_firstclaim'
    
    with open(firstclaim_file, "r") as file2:
        # Read each line in the file, readlines() returns a list of lines
        firstclaim = file2.readlines()
        # Combine the lines in the list into a string
        firstclaim = "".join(firstclaim)
    
    name_dict["inputs"].append(summary)
    name_dict["targets"].append(firstclaim)
    
    count += 1
    if count >= 100:
        break
        

In [12]:
df = pd.DataFrame(name_dict)

#print(df)

In [13]:
header = ["inputs", "targets"]

df.to_csv('val_10k_output.csv', columns=header, index=False)

csv = pd.read_csv("val_10k_output.csv").values
with tf.io.TFRecordWriter("pegasus/data/testdata/val_10k.tfrecords") as writer:
    for row in csv:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

In [14]:
header = ["inputs", "targets"]
df = pd.read_csv('val_10k_output.csv')

In [15]:
df

inputs  \
0    Generally, the present invention pertains to ...   
1    An object of the present invention in an inve...   
2    The foregoing objects are realized by the pre...   
3    If the transfer function of a power supply re...   
4    Therefore, the present invention aims at miti...   
..                                                ...   
95   The present invention resides in an EMI feedt...   
96   The exemplary embodiments of the present inve...   
97   The present invention provides such a method ...   
98   The present invention provides an industrial ...   
99   One object of the invention is to provide a b...   

                                              targets  
0   1. A controller comprising: \na programmable s...  
1   1. An inverted type front fork usable in a two...  
2   1. An apparatus for processing image data repr...  
3   1. A power supply comprising: \na switching co...  
4   1. A clearance control device for controlling ...  
..                                                ...  
95  1. A feedthrough terminal assembly for an acti...  
96  1. A rinse solution for removing at least one ...  
97  1. A method of integrating the operations of a...  
98  1. An equipment area network (EAN) for a plura...  
99  1. A bottle container provided with a protecti...  

[100 rows x 2 columns]